# Agrupamento

- Métodos não supervisionados (não requerem dados etiquetados)
- Objetivo: Encontrar subgrupos homogêneos de dados semelhantes (_clusters_).
- Aplicações: Segmentação de mercado (encontrar grupos de clientes com características semelhantes), análise de redes sociais (pessoas com comportamentos parecidos), etc.
- Não existe o conceito de overfitting, pois não existe target $y$.
- Existem métricas para medir o desempenho de uma clusterização

## 1. K-means
- Agrupar os dados em $K$ **non-overlapping** clusters
    - Os clusters são caracterizados por um ponto chamado **centroide**.
- Todos os pontos pertencem a algum cluster, e formam uma partição dos exemplos: **hard clustering**
- Algoritmo iterativo
- Precisa de normalização (usa distância euclidiana)
- Clusters esféricos

![k_means_intro](img/k_means_intro.png)

Na imagem acima, utilizou-se $K = 4$.

### 1.1 Algoritmo
Vamos supor que vamos começar o algoritmo com este dataset:

![k_means_example_1](img/k_means_example_1.png)

1. Inicialização de $K$ centroides:
- Probabilístico: tenta inicializar os centroides com uma abordagem probabilística, tenta garantir que os centroides vão começar distantes uns dos outros (K-means++)
- Aleatório

**Atenção:** os centroides podem ser qualquer ponto no espaço, não necessariamente do dataset.

![k_means_example_2](img/k_means_example_2.png)

2. Atribuição aos clusters:
- Nesta etapa, calculamos a distância euclidiana de cada ponto do dataset para cada cluster. Associamos cada ponto $x_i$ ao centroide mais próximo.
- Assim, construimos $K$ clusters iniciais:

![k_means_example_3](img/k_means_example_3.png)

3. Mover o centroide:
- Nesta etapa, cada centroide $\mu_j$ é movido para a **média** (dai a palavra _means_) dos pontos que foram atribuídos ao cluster correspondente a esse centroide.

$\mu_j^{(t+1)} = \frac{1}{|S_i^{(t)}|} \sum_{x_j \in S_i^{(t)}} x_j$

![k_means_example_4](img/k_means_example_4.png)

4. Executamos os passos 3 e 4 iterativamente, até a convergência:

![k_means_example_5](img/k_means_example_5.png)

### 1.2 Observações
- O resultado final **depende da inicialização dos $K$ centroides** no inicio do algoritmo. Ou seja, o algoritmo está sujeito a cair em mínimos locais de erro (_greedy_).
    - O que se faz pra evitar isso é utilizar o K-means++ pra tentar inicializar os centroides de forma inteligente
    - Também é possível rodar o algoritmo várias vezes com inicializações diferentes e escolher a inicialização com menor erro, porém pode ser computacionalmente custoso se o dataset for grande
- O K-means força todo ponto a pertencer a um cluster. Isso pode distorcer o resultado final, pois mesmo **outliers precisam pertencer a um cluster.** Como o K-means se baseia na média das observações atribuídas a um cluster, e a média é fortemente afetada por outliers, a posição do centroide também será influenciada.
- Por usar distâncias na atribuição de pontos a clusters, é necessário **normalizar os dados** para que a noção de distância não seja afetada pela magnitude das features.
- Curse of dimensionality: Se o número de features for grande, a noção de distância pode acabar se perdendo, e todos os pontos ficarem igualmente distantes.
- O algoritmo K-means minimiza a seguinte função objetivo: $J = \sum_{c=1}^k\sum_{x_j \in C_c} d(x_j, \bar{x}_c)^2$, onde $\bar{x}_c = \frac{1}{|C_c|}\sum_{x_j \in C_c} x_j$. Minimizar $J$ significa **minimizar as variâncias intra-cluster**.
- K-means funciona bem se os clusters são (hiper)esféricos, bem separados, de volumes aproximadamente iguais e possuem quantidades de pontos semelhantes


### 1.3 Variações

#### K-medianas
- Ao invés de usar as médias para calcular o centroide, usar a **mediana**
- Vantagem: menos sensível a outliers
- Desvantagem: implementação mais complexa (ordenação)

#### K-medoids (PAM = Partitioning around Medoids)
- Ao invés de usar as médias para calcular o centroide, usar o **medoide**
- O medoide é um ponto do dataset tomado como centroide, para o qual o grau de dissimilaridade em relação às outras observações é mínimo.
- O algoritmo é feito testando a troca dos medoids atuais por outros pontos do dataset e verificando qual disposição tem o menor erro.
- Vantagem: menos sensível a outliers, convergência assegurada com qualquer medida de dissimilaridade
- Desvantagem: complexidade quadrática com número de objetos

## 2. DBSCAN
- Agrupamento espacial baseado em densidade (Density-Based Clustering)
- A ideia principal é que um ponto pertence a um cluster se ele está **próximo de vários pontos daquele cluster**
- Nem todos os pontos precisam pertencer a um cluster: **soft clustering**

### 2.1 Parâmetros
- **eps**: distância que caracteriza uma vizinhança.
    - $A$ e $B$ são vizinhos se $d(A,B) \leq eps$
- minPts: número mínimo de pontos para definir um cluster.

### 2.2 Tipos de pontos
Os pontos podem ser classificados como:
- **Core point**: um ponto que está em uma região de raio **eps** onde há pelo menos **minPts** (incluindo ele mesmo)
- **Border/Edge point**: um ponto está na borda se ele faz parte de um cluster (está na vizinhança de um core point), mas não é um core point. 
- **Noise point (outlier)**: um ponto que não é nem um core point nem um border point.

![dbscan_intro](img/dbscan_intro.png)

### 2.3 Algoritmo
1. Selecionar um ponto de forma aleatória e uniforme do conjunto de dados

2. Verificar se o ponto selecionado é um **core point**, ou seja, se ele **contém pelo menos minpts na sua vizinhança de raio eps**

3. Classificar os pontos na vizinhança do core point. Eles serão necessariamente border points ou core points.

4. Os pontos que não estão na vizinhança de um core point são classificados como noise points.

O algoritmo termina quando todos os pontos foram explorados e classificados.

Esse algoritmo é melhor do que buscar por todos os core points no passo 1 (menor complexidade de armazenamento).

A escolha aleatória dos pontos no algoritmo não influencia a escolha dos core e noise points. Pode somente mudar a atribuição de pontos na borda de dois clusters, mas esses pontos não são importantes para o resultado final. 

### 2.4 Vantagens
- Robusto a **outliers**
- Ótimo para **separar clusters de alta densidade** de clusters de baixa densidade
- O número de clusters **não precisa ser especificado de antemão**
- Consegue encontrar clusters com **formatos arbitrários**
- Só precisa de dois parâmetros: eps e minpts

### 2.5 Desvantagens
- Não funciona bem para conjuntos de dados que possuem grupos com **grandes diferenças de densidade**
- Se a escala de dados não for conhecida, **determinar eps pode ser difícil**
- Depende da distância usada para determinar se um ponto está dentro de uma região de raio eps (precisa de normalização).
- Muito sensível aos parâmetros eps e minpts.

Exemplo: na figura abaixo observa-se que o DBScan não funciona muito bem quando a densidade dentro de um cluster é variável.

![dbscan_fail_variable_density](img/dbscan_fail_variable_density.png)

## 3. Algoritmos Hierárquicos

- Não exige que o número de clusters $k$ seja pré-especificado
- Resulta em uma representação em forma de árvore, chamada **dendograma**

![partitional_vs_hierarchical](img/partitional_vs_hierarchical.png)

- É importante definir uma **função distância** (similaridade) entre dois pontos:
    - Euclidian distance
    - Squared Euclidian distance
    - Manhattan distance
    - Maximum distance
    - Malahanobis distance
    - For text: Hamming distance, Cosine distance, Levenshtein distance
- também definimos um **critério de linkage** entre dois clusters (baseia-se na função distância):
    - Maximum or complete-linkage clustering
- Minimum or single-linkage clustering
- Unweighted average linkage clustering (or UPGMA)
- Weighted average linkage clustering (or WPGMA)
- Centroid linkage clustering, or UPGMC
- Minimum energy clustering

### 3.1 Aglomerativos x Divisivos

#### Aglomerativos

Nos métodos **aglomerativos** (também chamados de “bottom up”), cada observação começa dentro de seu próprio cluster, e **pares de clusters são combinados em um só** a medida em que subimos na hierarquia.

Algoritmo:
1. Definir uma métrica de dissimilaridade entre cada par de observações (geralmente usa-se distância euclidiana)
2. Começando da base do dendograma, cada uma das $n$ observações é tratada como sendo seu próprio cluster.
3. Os dois clusters mais similares entre eles são combinados em um só, e o processo é repetido até que todas as observações pertençam a um único cluster.

OBS:
- Podemos extrair informações sobre a similaridade de duas observações baseado na localização no **eixo vertical** onde os ramos da árvore contendo essas duas observações se unem pela primeira vez.

![dendogram_construction](img/dendogram_construction.png)

Notar que como o que importa é a posição vertical do elemento no dendograma, o elemento 6 está tão próximo de 4, quanto de 5, quanto de qualquer outro desse cluster (a distância horizontal no dendograma não importa!).

#### Divisivos
Nos métodos **divisivos** (“top down”), todas as observações começam em um cluster, e são realizadas **divisões recursivas** à medida em que descemos na hierarquia.

### 3.2 Tipos de Linkage
Mais usadas = average e complete

https://towardsdatascience.com/understanding-the-concept-of-hierarchical-clustering-technique-c6e8243758ec

#### Single link (MIN)
- Define a proximidade de clusters como sendo a **proximidade entre os pontos mais próximos** de cada cluster.
- “Minimal intercluster dissimilarity”
- Pode resultar em clusters longos e finos nos quais elementos próximos do mesmo cluster têm pequenas distâncias, mas elementos em extremidades opostas de um cluster podem estar muito mais distantes um do outro do que dois elementos de outros cluster.

![single_link](img/single_link.png)

#### Complete link (MAX)
- Define a proximidade de clusters como sendo a **proximidade entre os pontos mais distantes** de cada cluster.
- “Maximal intercluster dissimilarity”

![complete_link](img/complete_link.png)

#### Average
- Define a proximidade de clusters como sendo a **média de proximidade entre duplas de pontos** de cada cluster.
- “Mean intercluster dissimilarity”

![average_link](img/average_link.png)

#### Centroids
- Define a proximidade de clusters como sendo a **proximidade entre as centroides** de cada cluster.

![centroids_link](img/centroids_link.png)

- Não é monotônica, ou seja, a similaridade pode **aumentar** depois de um passo, podendo resultar em **inversões**, conforme a figura abaixo:

![dendogram_inversions](img/dendogram_inversions.png)

### 3.3 Escolhendo o número de clusters
Olhando o dendograma, podemos escolher o número de clusters vendo a maior distância vertical sem nenhuma linha horizontal passando por ela. No caso abaixo, temos 5 clusters.

![dendogram_splitted](img/dendogram_splitted.png)

## 4. Estratégias de definição do número de clusters
- Dependendo do domínio
A quantidade de clusters pode ser feita dependendo do domínio do problema. Se já conhecemos em quantos grupos queremos dividir, devemos utilizar esse valor. Muitas vezes é uma decisão de negócio.
- Elbow method
Se não temos ideia de quantos clusters deve ter o nosso resultado final, podemos utilizar o método do cotovelo.
Esse método consiste em aplicar o algoritmo de clusterização várias vezes, modificando o valor de $K$ e calculando a métrica de erro relacionada. Tomamos o valor de $K$ a partir do qual não percebemos uma grande queda do erro:

![elbow_method](img/elbow_method.png)

Neste caso, tomamos $K = 3$, pois para $K > 3$ o erro cai pouco conforme aumentamos a quantidade de clusters.

Problema: Nem sempre é perceptível essa variação na derivada do erro conforme aumentamos o valor de $K$:

![elbow_method_hard_decision](img/elbow_method_hard_decision.png)

Neste caso, talvez tomar $K = 5$ seja uma boa opção. Mas seria bom avaliar $K = 3$.

## 5. Mixture Models
Trata-se de um modelo que tenta identificar várias subpopulações em um mesmo dataset.

- Não supervisionado
- Modelo particional **com sobreposição**

### 5.1 Gaussian Mixture Models
Exemplo: se considerarmos as alturas de homens e mulheres separadamente, ambas as variáveis seguem uma distribuição normal. Se juntarmos as duas populações, teremos uma distribuição que é a soma das ocorrências dos valores de altura nas duas populações, mas com duas modas distintas, uma referente ao homens, e outra referente às mulheres. É esse tipo de distribuição conjunta que o modelo GMM tenta identificar e dissociar.

Um indício de que os dados seguem mixture model é quando os dados têm aparência multimodal, ou seja, existe mais de um pico na distribuição de dados.

#### Expectation Minimization
É um procedimento genérico para modelagem **probabilística de dados**.
De modo geral, a aprendizagem de modelos  costuma ser feita utilizando técnicas de **estimação por máxima verossimilhança**, porém no caso de mixture models, achar a solução de maior verossimilhança diferenciando a log likelihood e igualando a 0 é analiticamente impossível.

- É uma técnica numérica para estimação de máxima verossimilhança. OBS: Maximizar a verossimilhança pode ser visto como maximizar a compatibilidade entre as $N$ observações e o modelo
- É um método iterativo
- Vantagem: a máxima verossimilhança dos dados é estritamente crescente a cada iteração (garantia de que irá atingir um máximo local).
- Otimiza os parâmetros de uma função de distribuição de probabilidades de forma que esta represente os dados da forma mais verossímil possível

É representada pela função de distribuição de probabilidades abaixo:

$p(x) = \sum_{k=1}^K \phi_i \mathcal{N}(x|\mu_i, \sigma_i)$, onde:
-  $\phi_i$ é o peso do componente gaussiano
- $K$ é o número de componentes
- $\mu_i$ é a média do componente gaussiano
- $\sigma_i$ é a variância do componente gaussiano
- $\mathcal{N}(x|\mu_i, \sigma_i) = \frac{1}{\sqrt{2\pi\sigma_i}}e^{-\frac{(x - \mu_i)^2}{2\sigma_i^2}}$

![gmm_two_subpopulations](img/gmm_two_subpopulations.png)

#### O algoritmo
Passo 1: Inicialização
- Escolher aleatoriamente exemplos (sem repetição) do dataset $X = {x_1, …, x_N}$ para inicializar as estimativas de média $\hat{\mu}_1, …, \hat{\mu}_K$.
- Inicializar as estimativas de variância como sendo a variância da amostra: $\hat{\sigma}_1^2, …, \hat{\sigma}_K^2 = \frac{1}{N}\sum_{i=1}^N(x_i - \bar{x})^2$, onde $\bar{x} = \frac{1}{N}\sum_{i=1}^N x_i$ (média da amostra).
- Setar as estimativas das distribuições a priori dos componentes como sendo uma distribuição uniforme, ou seja, $\hat{\phi}_1, …, \hat{\phi}_K = \frac{1}{K}$

Passo 2: Expectation step
- Calcular a probabilidade a posteriori ($\forall i,k$)
$\hat{\gamma_{ik}} = \frac{\hat{\phi}_k\mathcal{N}(x_i|\hat{\mu}_k, \hat{\sigma}_k)}{\sum_{j=1}^K\hat{\phi}_j \mathcal{N}(x_i|\hat{\mu}_k, \hat{\sigma}_k)}$

Onde $\hat{\gamma_{ik}}$ é a probabilidade de que $x_i$ foi gerado pelo componente $C_k$, ou seja, $\hat{\gamma_{ik}} = p(C_k|x_i, \hat{\phi}, \hat{\mu}, \hat{\sigma})$

Passo 3: Maximization step
- Ajustar os parâmetros do modelo para maximizar a verossimilhança:
- $\hat{\phi}_k = \sum_{i=1}^N\frac{\hat{\gamma}_{ik}}{N}$
- $\hat{\mu}_k = \frac{\sum_{i=1}^N \hat{\gamma}_{ik}x_i}{\sum_{i=1}^N \hat{\gamma}_{ik}}$
- $\hat{\sigma}_k^2 = \frac{\sum_{i=1}^N \hat{\gamma}_{ik}(x_i - \hat{\mu}_k)^2}{\sum_{i=1}^N \hat{\gamma}_{ik}}$

Passo 4: Avaliação do critério de parada (ex: função de log-verossimilhança)

Passo 5: Interrupção ou retorno ao passo 2.    

![gmm_algorithm](img/gmm_algorithm.png)

#### EM x k-Means
- EM produz informações mais ricas sobre os dados (probabilidades associadas a cada cluster)
- A partição soft pode ser transformada em hard a partir das probabilidades extraídas do modelo
- EM possui um maior custo computacional (inversa das matrizes de covariância)
- k-Means é um caso particular de EM
- k-Means busca $k$ que minimiza $(x - \mu_k)^2$
- EM busca $k$ que minimiza $\frac{(x - \mu_k)^2}{\sigma^2}$ (leva em conta a variância)

## 6. Índices de validação

### 6.1 Internos
Avaliam a qualidade de um agrupamento de dados com base **apenas nos dados originais**.

#### Erro quadrático

$J = \sum_{i=1}^k\sum_{x_j \in C_i} d(x_j, \bar{x}_i)^2$

### 6.2 Relativos
Realizam uma análise comparativa entre diversos agrupamentos para definir qual deles é melhor sob algum aspecto. 

#### Silhueta

Métrica utilizada para medir a qualidade do agrupamento em termos de:
- **Separabilidade** entre clusters
- **Similaridade** de um objeto em relação ao seu cluster (**coesão**), quando comparado com outros clusters.

A silhueta é um valor que vai de -1 a 1, onde um valor elevado indica que a observação corresponde bem ao seu próprio cluster, e mal aos clusters vizinhos.

Para um ponto $i \in C_i$: $a(i) = \frac{1}{|C_i| - 1}\sum_{j \in C_i, i \neq j} d(i,j)$ (distância média de $i$ em relação a todos os outros pontos de seu cluster)

Isso é, $a(i)$ mede o quão boa é a atribuição de $i$ ao seu cluster (quanto menor o valor de $a$, melhor).

Para um ponto $i \in C_i$: $b(i) = min_{k \neq i}\frac{1}{|C_k|} \sum_{j \in C_k} d(i,j)$ (distância média de $i$ em relação a todos pontos de um cluster vizinho)

Ou seja, $b(i)$ é uma medida da dissimilaridade média de $i$ em relação ao cluster vizinho $C_k$.

A silhueta pode ser definida como : $s(i) = \frac{b(i) - a(i)}{max\{a(i),b(i)\}}$, se $|C_i| > 1$ e $s(i) = 0$, se $|C_i| = 1$

OBS: Existe também a **silhueta simplificada**, na qual $a(i)$ e $b(i)$ são calculados como a distância da observação $i$ ao **centróide do cluster em questão**.

===

Nesses métodos, utilizamos para todos os exemplos uma métrica que considera a similaridade entre um exemplo e o cluster no qual ele está (coesão), e também a similaridade com os clusters vizinhos (separação).
- Silhueta
A silhueta é uma métrica $s(t) \in [-1, +1]$.
- Se está próxima de +1, significa que o exemplo corresponde bem ao cluster em que ele se encontra.
- Se está próximo de -1, significa que corresponde melhor aos clusters vizinhos. - Se está próximo de 0, significa que o exemplo se encontra na borda de dois clusters.
Calculamos $s(t)$ para cada ponto do dataset e agrupamos os pontos segundo os clusters em que os pontos estão, em ordem decrescente do $s(t)$, como neste gráfico:

![silhouette_plot](img/silhouette_plot.png)

A ideia é que se os pontos do cluster corresponderem bem ao cluster em que estão, a silhueta do cluster ficará com valores parecidos e positivos, significando que o cluster faz sentido, e os exemplos correspondem bem à categoria à qual foram atribuídos (ver cluster azul).

Ao contrário, no cluster verde vemos que temos os exemplos “dolphin” e “porpoise” que são negativos no cluster dos mamíferos. Isso mostra que esses exemplos correspondem mais aos cluster vizinhos.

O calculo do $s(t)$ é feito assim:

$s(t) = \frac{b(i) - a(i)}{max\{a(i), b(i)\}}$, se $|C_i| > 1$

$s(t) = 0$, se $|C_i| = 1$

Com $a(i)$ sendo uma medida de dissimilaridade do exemplo $x_i$ em relação ao próprio cluster, e $b(i)$ sendo uma medida de quão dissimilar $x_i$ é em relação aos clusters vizinhos.

Assim, $s(t) \simeq 1$ se $a(t) \ll b(t)$, ou seja, se $x_i$ for muito dissimilar com os clusters vizinhos, e mais similar ao próprio cluster.

### 6.3 Externos
Avaliam o agrupamento em relação a uma estrutura pré-especificada.

São definidas duas partições:
- $P_r$ = partição de referência (real) - **golden truth**
- $P_a$ = partição sob avaliação (obtida pelo modelo)

Chamamos de **clusters** os grupos obtidos pela partição $P_a$ e de **classes** os agrupamentos reais provenientes de $P_r$.

#### Índice de Rand
O índice de Rand é a probabilidade de que dois pontos pertençam ao mesmo cluster para as partições $P_r$ e $P_a$.

$RI = \frac{a + d}{a + b + c + d}$, onde:
- $a$: número de pares da mesma classe e do mesmo cluster 
- $b$: número de pares da mesma classe e de clusters distintos
- $c$: número de pares de classes distintas e do mesmo cluster
- $d$: número de pares de classes e clusters distintos

Ex: na figura abaixo, temos:
- 2 classes (círculos e quadrados)
- 3 clusters (preto, branco e cinza)
- $a = 5$; $b = 7$; $c = 2$; $d = 14$

$RI = \frac{5 + 14}{5+7+2+14} = 0.6785$

![rand_index](img/rand_index.png)

OBS: favorece partições com mais grupos, pois:
- atribui o mesmo peso para objetos agregados ($a$) ou separados ($d$)
- $d$ tende a dominar o índice
- quanto mais grupos, maior $d$

#### Índice de Jaccard
Elimina o $d$ do índice de Rand, considerando que a separabilidade dos objetos é apenas uma consequência de um bom agrupamento.

$J_c = \frac{a}{a + b + c}$

Ex: no exemplo anterior, temos:
$RI = \frac{5}{5+7+2} = 0.3571$